<p style="text-align:center;">
    <img src="https://raw.githubusercontent.com/skyplane-project/skyplane/main/docs/_static/logo-light-mode.png" width=500>
</p>

# Welcome to Skyplane!

## Skyplane enables fast data transfers between any cloud

Skyplane is a tool for blazingly fast bulk data transfers between object stores in the cloud. It provisions a fleet of VMs in the cloud to transfer data in parallel while using compression and bandwidth tiering to reduce cost.

Skyplane is:
1. 🔥 Blazing fast ([110x faster than AWS DataSync](https://skyplane.org/en/latest/benchmark.html))
2. 🤑 Cheap (4x cheaper than rsync)
3. 🌐 Universal (AWS, Azure and GCP)

You can use Skyplane to transfer data: 
* between object stores within a cloud provider (e.g. AWS us-east-1 to AWS us-west-2)
* between object stores across multiple cloud providers (e.g. AWS us-east-1 to GCP us-central1)
* between local storage and cloud object stores (experimental)

# Learning outcomes 🎯

After completing this notebook, you would have:

1. An understand of unix-inspired Skyplane interface.
2. Transfered imagenet data from US-EAST-1 (N. Virginia) to EU-WEST-1 (Ireland).
3. Compare and contrast `aws s3 cp` with `skyplane cp`
4. Terminate the transfer and clean up state.

# How to use this Tutorial

These notebooks serve as a guide to Skyplane. At any point if you happen to get stuck, feel free to ping us on `#skyplane` channel on the [Skycamp slack.](https://join.slack.com/t/skycamp2022/shared_invite/zt-1gsrgky1z-iSFVEEOMSUD7Dd7B5syCsA)

We will describe what we are doing in this notebook. The commands and the example response are included. We highly recommend you open an terminal and run commands yourself. These points will be highlighted with this **icon**:

### 💻 - Run commands in an interactive terminal window

You can use this icon as a hint to know when to switch away from the current notebook and edit a file or open a terminal. We also have example outputs that you can use to ensure consistency. 

> **💡 Hint** - If you're using jupyter lab, you can create a terminal in your browser by going to `File -> New -> Terminal`

# Preflight  - Initializing cloud credentials

Before we start this tutorial, it is important to configure skyplane with your AWS credentials. For this tutorial, have already preloaded your VMs with a default credential. 

> **💡 Hint** - If you run into any issues, please contact one of the Skyplane team members immediately. This step is critical to follow through the tutorial.

💻 Configure Skyplane with AWS credentials. Run the below command in your terminal and choose `Y` only for AWS, and `n` for GCP and Azure.

`$skyplane init`

```
 _____ _   ____   _______ _       ___   _   _  _____ 
/  ___| | / /\ \ / / ___ \ |     / _ \ | \ | ||  ___|
\ `--.| |/ /  \ V /| |_/ / |    / /_\ \|  \| || |__  
 `--. \    \   \ / |  __/| |    |  _  || . ` ||  __| 
/\__/ / |\  \  | | | |   | |____| | | || |\  || |___ 
\____/\_| \_/  \_/ \_|   \_____/\_| |_/\_| \_/\____/

03:37:54 [DEBUG] Found existing configuration file at /root/.skyplane/config, 
loading

(1) Configuring AWS:
    Do you want to configure AWS support in Skyplane? [Y/n]:
    Loaded AWS credentials from the AWS CLI [IAM access key ID: ...ZEXYJW]
    AWS region config file saved to /root/.skyplane/aws_config

(2) Configuring Azure:
    Do you want to configure Azure support in Skyplane? [Y/n]: n
    Disabling Azure support

(3) Configuring GCP:
    Do you want to configure GCP support in Skyplane? [Y/n]: n
    Disabling Google Cloud support

Config file saved to /root/.skyplane/config
To disable performance logging info: 
https://skyplane.org/en/latest/performance_stats_collection.html
```




# Exercise: Transferring Data with Skyplane

The core of Skyplane is based around the `cp` command. Suppose you are working with the Stable Diffusion neural network. You’ve just gotten a fresh batch of data, but it is in us-east-1, while your network is in eu-west-1. Skyplane can help you efficiently transfer this data over to wherever your neural network may be so that you can train your model. Let’s prepare for a transfer by first initializing buckets in a few different cloud regions in AWS.

# Creating Buckets

### Setting up AWS in the Source region

First, let’s create a bucket in your source AWS region to store the data for the neural network.

💻 `aws s3 mb s3://skycamp-[name]-diffusion-data --region eu-west-1`
```
make_bucket: skycamp-[name]-diffusion-data
```

### Setting up AWS S3 in the destination region

Let’s also create the bucket in your destination AWS region to store a partial subset of the data. 

💻 `aws s3 mb s3://skycamp-[name]-diffusion-backup-data --region eu-west-1`

```
make_bucket: skycamp-[name]-diffusion-backup-data
```

Now, run this command to copy over the starter data for this bucket. 

💻 `aws s3 cp --recursive s3://stable-diffusion-partial-data-eu-west-1 s3://skycamp-[name]-diffusion-backup-data`

```
copy: s3://stable-diffusion-partial-data-eu-west-1/part-00006-5b54c5d5-bbcf-484d-a2ce-0d6f73df1a36-c000.snappy.parquet to s3://skycamp-[name]-backup-data/part-00006-5b54c5d5-bbcf-484d-a2ce-0d6f73df1a36-c000.snappy.parquet
copy: s3://stable-diffusion-partial-data-eu-west-1/part-00004-5b54c5d5-bbcf-484d-a2ce-0d6f73df1a36-c000.snappy.parquet to s3://skycamp-[name]-backup-data/part-00004-5b54c5d5-bbcf-484d-a2ce-0d6f73df1a36-c000.snappy.parquet
copy: s3://stable-diffusion-partial-data-eu-west-1/part-00007-5b54c5d5-bbcf-484d-a2ce-0d6f73df1a36-c000.snappy.parquet to s3://skycamp-[name]-backup-data/part-00007-5b54c5d5-bbcf-484d-a2ce-0d6f73df1a36-c000.snappy.parquet
copy: s3://stable-diffusion-partial-data-eu-west-1/part-00001-5b54c5d5-bbcf-484d-a2ce-0d6f73df1a36-c000.snappy.parquet to s3://skycamp-[name]-backup-data/part-00001-5b54c5d5-bbcf-484d-a2ce-0d6f73df1a36-c000.snappy.parquet
copy: s3://stable-diffusion-partial-data-eu-west-1/part-00013-5b54c5d5-bbcf-484d-a2ce-0d6f73df1a36-c000.snappy.parquet to s3://skycamp-[name]-backup-data/part-00013-5b54c5d5-bbcf-484d-a2ce-0d6f73df1a36-c000.snappy.parquet
copy: s3://stable-diffusion-partial-data-eu-west-1/part-00003-5b54c5d5-bbcf-484d-a2ce-0d6f73df1a36-c000.snappy.parquet to s3://skycamp-[name]-backup-data/part-00003-5b54c5d5-bbcf-484d-a2ce-0d6f73df1a36-c000.snappy.parquet
copy: s3://stable-diffusion-partial-data-eu-west-1/part-00011-5b54c5d5-bbcf-484d-a2ce-0d6f73df1a36-c000.snappy.parquet to s3://skycamp-[name]-backup-data/part-00011-5b54c5d5-bbcf-484d-a2ce-0d6f73df1a36-c000.snappy.parquet
copy: s3://stable-diffusion-partial-data-eu-west-1/part-00009-5b54c5d5-bbcf-484d-a2ce-0d6f73df1a36-c000.snappy.parquet to s3://skycamp-[name]-backup-data/part-00009-5b54c5d5-bbcf-484d-a2ce-0d6f73df1a36-c000.snappy.parquet
copy: s3://stable-diffusion-partial-data-eu-west-1/part-00010-5b54c5d5-bbcf-484d-a2ce-0d6f73df1a36-c000.snappy.parquet to s3://skycamp-[name]-backup-data/part-00010-5b54c5d5-bbcf-484d-a2ce-0d6f73df1a36-c000.snappy.parquet
copy: s3://stable-diffusion-partial-data-eu-west-1/part-00017-5b54c5d5-bbcf-484d-a2ce-0d6f73df1a36-c000.snappy.parquet to s3://skycamp-[name]-backup-data/part-00017-5b54c5d5-bbcf-484d-a2ce-0d6f73df1a36-c000.snappy.parquet


```


# Copying data between AWS regions

## Transferring data between AWS regions with aws s3 cp

Each cloud provider has dedicated tools to move data between cloud regions. Let’s try transferring over the data using AWS’s built in cp command:

💻 `aws s3 cp --recursive s3://stable-diffusion-data s3://skycamp-[name]-diffusion-data`

```
copy: s3://stable-diffusion-data/part-00000-5b54c5d5-bbcf-484d-a2ce-0d6f73df1a36-c000.snappy.parquet to s3://skycamp-anton-diffusion-data/part-00000-5b54c5d5-bbcf-484d-a2ce-0d6f73df1a36-c000.snappy.parquet
copy: s3://stable-diffusion-data/part-00002-5b54c5d5-bbcf-484d-a2ce-0d6f73df1a36-c000.snappy.parquet to s3://skycamp-anton-diffusion-data/part-00002-5b54c5d5-bbcf-484d-a2ce-0d6f73df1a36-c000.snappy.parquet
copy: s3://stable-diffusion-data/part-00005-5b54c5d5-bbcf-484d-a2ce-0d6f73df1a36-c000.snappy.parquet to s3://skycamp-anton-diffusion-data/part-00005-5b54c5d5-bbcf-484d-a2ce-0d6f73df1a36-c000.snappy.parquet
copy: s3://stable-diffusion-data/part-00008-5b54c5d5-bbcf-484d-a2ce-0d6f73df1a36-c000.snappy.parquet to s3://skycamp-anton-diffusion-data/part-00008-5b54c5d5-bbcf-484d-a2ce-0d6f73df1a36-c000.snappy.parquet
copy: s3://stable-diffusion-data/part-00001-5b54c5d5-bbcf-484d-a2ce-0d6f73df1a36-c000.snappy.parquet to s3://skycamp-anton-diffusion-data/part-00001-5b54c5d5-bbcf-484d-a2ce-0d6f73df1a36-c000.snappy.parquet
Completed 4.2 GiB/~15.7 GiB (36.9 MiB/s) with ~12 file(s) remaining (calculating...)

```

This will take a long time to complete. Feel free to interrupt the command. Notice that it copies data at under 25 MiB/s.


## Transferring data between AWS regions with skyplane cp

Let’s delete the data in the bucket:

💻 `aws s3 rm --recursive s3://skycamp-[name]-diffusion-data`

```
delete: s3://skycamp-anton-diffusion-data/part-00002-5b54c5d5-bbcf-484d-a2ce-0d6f73df1a36-c000.snappy.parquet
delete: s3://skycamp-anton-diffusion-data/part-00005-5b54c5d5-bbcf-484d-a2ce-0d6f73df1a36-c000.snappy.parquet
delete: s3://skycamp-anton-diffusion-data/part-00000-5b54c5d5-bbcf-484d-a2ce-0d6f73df1a36-c000.snappy.parquet
delete: s3://skycamp-anton-diffusion-data/part-00003-5b54c5d5-bbcf-484d-a2ce-0d6f73df1a36-c000.snappy.parquet
delete: s3://skycamp-anton-diffusion-data/part-00008-5b54c5d5-bbcf-484d-a2ce-0d6f73df1a36-c000.snappy.parquet
delete: s3://skycamp-anton-diffusion-data/part-00006-5b54c5d5-bbcf-484d-a2ce-0d6f73df1a36-c000.snappy.parquet
delete: s3://skycamp-anton-diffusion-data/part-00004-5b54c5d5-bbcf-484d-a2ce-0d6f73df1a36-c000.snappy.parquet
delete: s3://skycamp-anton-diffusion-data/part-00009-5b54c5d5-bbcf-484d-a2ce-0d6f73df1a36-c000.snappy.parquet
delete: s3://skycamp-anton-diffusion-data/part-00001-5b54c5d5-bbcf-484d-a2ce-0d6f73df1a36-c000.snappy.parquet
delete: s3://skycamp-anton-diffusion-data/part-00012-5b54c5d5-bbcf-484d-a2ce-0d6f73df1a36-c000.snappy.parquet
delete: s3://skycamp-anton-diffusion-data/part-00014-5b54c5d5-bbcf-484d-a2ce-0d6f73df1a36-c000.snappy.parquet
delete: s3://skycamp-anton-diffusion-data/part-00013-5b54c5d5-bbcf-484d-a2ce-0d6f73df1a36-c000.snappy.parquet
delete: s3://skycamp-anton-diffusion-data/part-00015-5b54c5d5-bbcf-484d-a2ce-0d6f73df1a36-c000.snappy.parquet
delete: s3://skycamp-anton-diffusion-data/part-00007-5b54c5d5-bbcf-484d-a2ce-0d6f73df1a36-c000.snappy.parquet
delete: s3://skycamp-anton-diffusion-data/part-00016-5b54c5d5-bbcf-484d-a2ce-0d6f73df1a36-c000.snappy.parquet
delete: s3://skycamp-anton-diffusion-data/part-00018-5b54c5d5-bbcf-484d-a2ce-0d6f73df1a36-c000.snappy.parquet
delete: s3://skycamp-anton-diffusion-data/part-00011-5b54c5d5-bbcf-484d-a2ce-0d6f73df1a36-c000.snappy.parquet
delete: s3://skycamp-anton-diffusion-data/part-00020-5b54c5d5-bbcf-484d-a2ce-0d6f73df1a36-c000.snappy.parquet
delete: s3://skycamp-anton-diffusion-data/part-00019-5b54c5d5-bbcf-484d-a2ce-0d6f73df1a36-c000.snappy.parquet
delete: s3://skycamp-anton-diffusion-data/part-00022-5b54c5d5-bbcf-484d-a2ce-0d6f73df1a36-c000.snappy.parquet
delete: s3://skycamp-anton-diffusion-data/part-00017-5b54c5d5-bbcf-484d-a2ce-0d6f73df1a36-c000.snappy.parquet
delete: s3://skycamp-anton-diffusion-data/part-00021-5b54c5d5-bbcf-484d-a2ce-0d6f73df1a36-c000.snappy.parquet
delete: s3://skycamp-anton-diffusion-data/part-00025-5b54c5d5-bbcf-484d-a2ce-0d6f73df1a36-c000.snappy.parquet
delete: s3://skycamp-anton-diffusion-data/part-00010-5b54c5d5-bbcf-484d-a2ce-0d6f73df1a36-c000.snappy.parquet
delete: s3://skycamp-anton-diffusion-data/part-00026-5b54c5d5-bbcf-484d-a2ce-0d6f73df1a36-c000.snappy.parquet
delete: s3://skycamp-anton-diffusion-data/part-00023-5b54c5d5-bbcf-484d-a2ce-0d6f73df1a36-c000.snappy.parquet
delete: s3://skycamp-anton-diffusion-data/part-00028-5b54c5d5-bbcf-484d-a2ce-0d6f73df1a36-c000.snappy.parquet
delete: s3://skycamp-anton-diffusion-data/part-00024-5b54c5d5-bbcf-484d-a2ce-0d6f73df1a36-c000.snappy.parquet
delete: s3://skycamp-anton-diffusion-data/part-00029-5b54c5d5-bbcf-484d-a2ce-0d6f73df1a36-c000.snappy.parquet
delete: s3://skycamp-anton-diffusion-data/part-00030-5b54c5d5-bbcf-484d-a2ce-0d6f73df1a36-c000.snappy.parquet
delete: s3://skycamp-anton-diffusion-data/part-00027-5b54c5d5-bbcf-484d-a2ce-0d6f73df1a36-c000.snappy.parquet
delete: s3://skycamp-anton-diffusion-data/part-00031-5b54c5d5-bbcf-484d-a2ce-0d6f73df1a36-c000.snappy.parquet

```


We will now redo this transfer, but this time by using Skyplane. Compare the time that it takes!

💻 `skyplane cp -r s3://stable-diffusion-data-us-east-1 s3://skycamp-[name]-diffusion-data`

```
 _____ _   ____   _______ _       ___   _   _  _____
/  ___| | / /\ \ / / ___ \ |     / _ \ | \ | ||  ___|
\ `--.| |/ /  \ V /| |_/ / |    / /_\ \|  \| || |__
 `--. \    \   \ / |  __/| |    |  _  || . ` ||  __|
/\__/ / |\  \  | | | |   | |____| | | || |\  || |___
\____/\_| \_/  \_/ \_|   \_____/\_| |_/\_| \_/\____/


Will transfer 32 objects totaling 35.90GB from aws:us-east-1 to aws:us-east-1
    VMs to provision: 1x aws:us-east-1
    part-00000-5b54c5d5-bbcf-484d-a2ce-0d6f73df1a36-c000.snappy.parquet => part-00000-5b54c5d5-bbcf-484d-a2ce-0d6f73df1a36-c000.snappy.parquet
    part-00001-5b54c5d5-bbcf-484d-a2ce-0d6f73df1a36-c000.snappy.parquet => part-00001-5b54c5d5-bbcf-484d-a2ce-0d6f73df1a36-c000.snappy.parquet
    part-00002-5b54c5d5-bbcf-484d-a2ce-0d6f73df1a36-c000.snappy.parquet => part-00002-5b54c5d5-bbcf-484d-a2ce-0d6f73df1a36-c000.snappy.parquet
    part-00003-5b54c5d5-bbcf-484d-a2ce-0d6f73df1a36-c000.snappy.parquet => part-00003-5b54c5d5-bbcf-484d-a2ce-0d6f73df1a36-c000.snappy.parquet
    ...
    part-00028-5b54c5d5-bbcf-484d-a2ce-0d6f73df1a36-c000.snappy.parquet => part-00028-5b54c5d5-bbcf-484d-a2ce-0d6f73df1a36-c000.snappy.parquet
    part-00029-5b54c5d5-bbcf-484d-a2ce-0d6f73df1a36-c000.snappy.parquet => part-00029-5b54c5d5-bbcf-484d-a2ce-0d6f73df1a36-c000.snappy.parquet
    part-00030-5b54c5d5-bbcf-484d-a2ce-0d6f73df1a36-c000.snappy.parquet => part-00030-5b54c5d5-bbcf-484d-a2ce-0d6f73df1a36-c000.snappy.parquet
    part-00031-5b54c5d5-bbcf-484d-a2ce-0d6f73df1a36-c000.snappy.parquet => part-00031-5b54c5d5-bbcf-484d-a2ce-0d6f73df1a36-c000.snappy.parquet
Continue? [Y/n]: Y
Transfer starting (Tip: Enable auto-confirmation with `skyplane config set autoconfirm true`)

Storing debug information for transfer in /tmp/skyplane/transfer_logs/20221018_205648/client.log
✓ Initializing cloud keys (3/3) in 2.84s
✓ Provisioning gateway instances (1/1) in 34.48s
✓ Installing gateway package (1/1) in 12.41s
🚀 35.90GB transfer job launched
  Transfer progress (completing multi-part uploads) ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 35.9/35.9 GiB 602.0 MB/s 0:00:00
✓ Deprovisioning instances (1/1) in 2.42s
⠙ Verifying all files were copied

✅ Transfer completed successfully
Transfer runtime: 61.94s, Throughput: 4.64Gbps

```


# Syncing data between AWS regions
Now, let’s suppose that you have a bucket storing a backup of your dataset, and it already has some of the data there.


## Syncing with aws s3 sync

Let’s try using AWS’s built-in sync command to update the weights.

💻 `aws s3 sync s3://skycamp-[name]-diffusion-data s3://skycamp-[name]-diffusion-backup-data`

```
copy: s3://skycamp-anton-diffusion-data/part-00000-5b54c5d5-bbcf-484d-a2ce-0d6f73df1a36-c000.snappy.parquet to s3://skycamp-anton-diffusion-backup-data/part-00000-5b54c5d5-bbcf-484d-a2ce-0d6f73df1a36-c000.snappy.parquet
copy: s3://skycamp-anton-diffusion-data/part-00005-5b54c5d5-bbcf-484d-a2ce-0d6f73df1a36-c000.snappy.parquet to s3://skycamp-anton-diffusion-backup-data/part-00005-5b54c5d5-bbcf-484d-a2ce-0d6f73df1a36-c000.snappy.parquet
copy: s3://skycamp-anton-diffusion-data/part-00002-5b54c5d5-bbcf-484d-a2ce-0d6f73df1a36-c000.snappy.parquet to s3://skycamp-anton-diffusion-backup-data/part-00002-5b54c5d5-bbcf-484d-a2ce-0d6f73df1a36-c000.snappy.parquet
Completed 320.0 MiB/~13.4 GiB (36.9 MiB/s) with ~9 file(s) remaining (calculating...)

```


Make sure to record the time it took! 	Again, feel free to interrupt it once you’ve gotten a sense of the throughput. 


## Syncing with Skyplane sync

First, let’s reset the bucket so that we can try it out with Skyplane. 

💻 `aws s3 rm --recursive s3://skycamp-[name]-diffusion-backup-data`

```
delete: s3://skycamp-anton-diffusion-backup-data/part-00000-5b54c5d5-bbcf-484d-a2ce-0d6f73df1a36-c000.snappy.parquet
delete: s3://skycamp-anton-diffusion-backup-data/part-00002-5b54c5d5-bbcf-484d-a2ce-0d6f73df1a36-c000.snappy.parquet
delete: s3://skycamp-anton-diffusion-backup-data/part-00005-5b54c5d5-bbcf-484d-a2ce-0d6f73df1a36-c000.snappy.parquet
delete: s3://skycamp-anton-diffusion-backup-data/part-00003-5b54c5d5-bbcf-484d-a2ce-0d6f73df1a36-c000.snappy.parquet
delete: s3://skycamp-anton-diffusion-backup-data/part-00008-5b54c5d5-bbcf-484d-a2ce-0d6f73df1a36-c000.snappy.parquet
delete: s3://skycamp-anton-diffusion-backup-data/part-00001-5b54c5d5-bbcf-484d-a2ce-0d6f73df1a36-c000.snappy.parquet
delete: s3://skycamp-anton-diffusion-backup-data/part-00004-5b54c5d5-bbcf-484d-a2ce-0d6f73df1a36-c000.snappy.parquet
delete: s3://skycamp-anton-diffusion-backup-data/part-00007-5b54c5d5-bbcf-484d-a2ce-0d6f73df1a36-c000.snappy.parquet
delete: s3://skycamp-anton-diffusion-backup-data/part-00012-5b54c5d5-bbcf-484d-a2ce-0d6f73df1a36-c000.snappy.parquet
delete: s3://skycamp-anton-diffusion-backup-data/part-00006-5b54c5d5-bbcf-484d-a2ce-0d6f73df1a36-c000.snappy.parquet
delete: s3://skycamp-anton-diffusion-backup-data/part-00010-5b54c5d5-bbcf-484d-a2ce-0d6f73df1a36-c000.snappy.parquet
delete: s3://skycamp-anton-diffusion-backup-data/part-00014-5b54c5d5-bbcf-484d-a2ce-0d6f73df1a36-c000.snappy.parquet
delete: s3://skycamp-anton-diffusion-backup-data/part-00009-5b54c5d5-bbcf-484d-a2ce-0d6f73df1a36-c000.snappy.parquet
delete: s3://skycamp-anton-diffusion-backup-data/part-00015-5b54c5d5-bbcf-484d-a2ce-0d6f73df1a36-c000.snappy.parquet
delete: s3://skycamp-anton-diffusion-backup-data/part-00011-5b54c5d5-bbcf-484d-a2ce-0d6f73df1a36-c000.snappy.parquet
delete: s3://skycamp-anton-diffusion-backup-data/part-00013-5b54c5d5-bbcf-484d-a2ce-0d6f73df1a36-c000.snappy.parquet
delete: s3://skycamp-anton-diffusion-backup-data/part-00018-5b54c5d5-bbcf-484d-a2ce-0d6f73df1a36-c000.snappy.parquet
delete: s3://skycamp-anton-diffusion-backup-data/part-00017-5b54c5d5-bbcf-484d-a2ce-0d6f73df1a36-c000.snappy.parquet
delete: s3://skycamp-anton-diffusion-backup-data/part-00016-5b54c5d5-bbcf-484d-a2ce-0d6f73df1a36-c000.snappy.parquet
delete: s3://skycamp-anton-diffusion-backup-data/part-00020-5b54c5d5-bbcf-484d-a2ce-0d6f73df1a36-c000.snappy.parquet
delete: s3://skycamp-anton-diffusion-backup-data/part-00019-5b54c5d5-bbcf-484d-a2ce-0d6f73df1a36-c000.snappy.parquet
delete: s3://skycamp-anton-diffusion-backup-data/part-00024-5b54c5d5-bbcf-484d-a2ce-0d6f73df1a36-c000.snappy.parquet
delete: s3://skycamp-anton-diffusion-backup-data/part-00021-5b54c5d5-bbcf-484d-a2ce-0d6f73df1a36-c000.snappy.parquet
delete: s3://skycamp-anton-diffusion-backup-data/part-00023-5b54c5d5-bbcf-484d-a2ce-0d6f73df1a36-c000.snappy.parquet
delete: s3://skycamp-anton-diffusion-backup-data/part-00028-5b54c5d5-bbcf-484d-a2ce-0d6f73df1a36-c000.snappy.parquet
delete: s3://skycamp-anton-diffusion-backup-data/part-00022-5b54c5d5-bbcf-484d-a2ce-0d6f73df1a36-c000.snappy.parquet
delete: s3://skycamp-anton-diffusion-backup-data/part-00025-5b54c5d5-bbcf-484d-a2ce-0d6f73df1a36-c000.snappy.parquet
delete: s3://skycamp-anton-diffusion-backup-data/part-00026-5b54c5d5-bbcf-484d-a2ce-0d6f73df1a36-c000.snappy.parquet
delete: s3://skycamp-anton-diffusion-backup-data/part-00027-5b54c5d5-bbcf-484d-a2ce-0d6f73df1a36-c000.snappy.parquet
delete: s3://skycamp-anton-diffusion-backup-data/part-00029-5b54c5d5-bbcf-484d-a2ce-0d6f73df1a36-c000.snappy.parquet
delete: s3://skycamp-anton-diffusion-backup-data/part-00030-5b54c5d5-bbcf-484d-a2ce-0d6f73df1a36-c000.snappy.parquet
delete: s3://skycamp-anton-diffusion-backup-data/part-00031-5b54c5d5-bbcf-484d-a2ce-0d6f73df1a36-c000.snappy.parquet

```


💻 `aws s3 cp --recursive s3://stable-diffusion-partial-data s3://skycamp-[name]-diffusion-backup-data`

```
copy: s3://stable-diffusion-partial-data-eu-west-1/part-00006-5b54c5d5-bbcf-484d-a2ce-0d6f73df1a36-c000.snappy.parquet to s3://skycamp-[name]-backup-data/part-00006-5b54c5d5-bbcf-484d-a2ce-0d6f73df1a36-c000.snappy.parquet
copy: s3://stable-diffusion-partial-data-eu-west-1/part-00004-5b54c5d5-bbcf-484d-a2ce-0d6f73df1a36-c000.snappy.parquet to s3://skycamp-[name]-backup-data/part-00004-5b54c5d5-bbcf-484d-a2ce-0d6f73df1a36-c000.snappy.parquet
copy: s3://stable-diffusion-partial-data-eu-west-1/part-00007-5b54c5d5-bbcf-484d-a2ce-0d6f73df1a36-c000.snappy.parquet to s3://skycamp-[name]-backup-data/part-00007-5b54c5d5-bbcf-484d-a2ce-0d6f73df1a36-c000.snappy.parquet
copy: s3://stable-diffusion-partial-data-eu-west-1/part-00001-5b54c5d5-bbcf-484d-a2ce-0d6f73df1a36-c000.snappy.parquet to s3://skycamp-[name]-backup-data/part-00001-5b54c5d5-bbcf-484d-a2ce-0d6f73df1a36-c000.snappy.parquet
copy: s3://stable-diffusion-partial-data-eu-west-1/part-00013-5b54c5d5-bbcf-484d-a2ce-0d6f73df1a36-c000.snappy.parquet to s3://skycamp-[name]-backup-data/part-00013-5b54c5d5-bbcf-484d-a2ce-0d6f73df1a36-c000.snappy.parquet
copy: s3://stable-diffusion-partial-data-eu-west-1/part-00003-5b54c5d5-bbcf-484d-a2ce-0d6f73df1a36-c000.snappy.parquet to s3://skycamp-[name]-backup-data/part-00003-5b54c5d5-bbcf-484d-a2ce-0d6f73df1a36-c000.snappy.parquet
copy: s3://stable-diffusion-partial-data-eu-west-1/part-00011-5b54c5d5-bbcf-484d-a2ce-0d6f73df1a36-c000.snappy.parquet to s3://skycamp-[name]-backup-data/part-00011-5b54c5d5-bbcf-484d-a2ce-0d6f73df1a36-c000.snappy.parquet
copy: s3://stable-diffusion-partial-data-eu-west-1/part-00009-5b54c5d5-bbcf-484d-a2ce-0d6f73df1a36-c000.snappy.parquet to s3://skycamp-[name]-backup-data/part-00009-5b54c5d5-bbcf-484d-a2ce-0d6f73df1a36-c000.snappy.parquet
copy: s3://stable-diffusion-partial-data-eu-west-1/part-00010-5b54c5d5-bbcf-484d-a2ce-0d6f73df1a36-c000.snappy.parquet to s3://skycamp-[name]-backup-data/part-00010-5b54c5d5-bbcf-484d-a2ce-0d6f73df1a36-c000.snappy.parquet
copy: s3://stable-diffusion-partial-data-eu-west-1/part-00017-5b54c5d5-bbcf-484d-a2ce-0d6f73df1a36-c000.snappy.parquet to s3://skycamp-[name]-backup-data/part-00017-5b54c5d5-bbcf-484d-a2ce-0d6f73df1a36-c000.snappy.parquet


```

Now, let’s use Skyplane’s sync command to update the weights, and again compare the time that it takes. 


💻 `skyplane sync s3://skycamp-[name]-diffusion-data s3://skycamp-[name]-diffusion-backup-data`

```
 _____ _   ____   _______ _       ___   _   _  _____
/  ___| | / /\ \ / / ___ \ |     / _ \ | \ | ||  ___|
\ `--.| |/ /  \ V /| |_/ / |    / /_\ \|  \| || |__
 `--. \    \   \ / |  __/| |    |  _  || . ` ||  __|
/\__/ / |\  \  | | | |   | |____| | | || |\  || |___
\____/\_| \_/  \_/ \_|   \_____/\_| |_/\_| \_/\____/


Will transfer 22 objects totaling 20.16GB from aws:us-east-1 to aws:us-west-1
    VMs to provision: 1x aws:us-east-1, 1x aws:us-west-1
    Estimated egress cost: $0.40 at $0.02/GB
    part-00000-5b54c5d5-bbcf-484d-a2ce-0d6f73df1a36-c000.snappy.parquet => part-00000-5b54c5d5-bbcf-484d-a2ce-0d6f73df1a36-c000.snappy.parquet
    part-00002-5b54c5d5-bbcf-484d-a2ce-0d6f73df1a36-c000.snappy.parquet => part-00002-5b54c5d5-bbcf-484d-a2ce-0d6f73df1a36-c000.snappy.parquet
    part-00005-5b54c5d5-bbcf-484d-a2ce-0d6f73df1a36-c000.snappy.parquet => part-00005-5b54c5d5-bbcf-484d-a2ce-0d6f73df1a36-c000.snappy.parquet
    part-00008-5b54c5d5-bbcf-484d-a2ce-0d6f73df1a36-c000.snappy.parquet => part-00008-5b54c5d5-bbcf-484d-a2ce-0d6f73df1a36-c000.snappy.parquet
    ...
    part-00028-5b54c5d5-bbcf-484d-a2ce-0d6f73df1a36-c000.snappy.parquet => part-00028-5b54c5d5-bbcf-484d-a2ce-0d6f73df1a36-c000.snappy.parquet
    part-00029-5b54c5d5-bbcf-484d-a2ce-0d6f73df1a36-c000.snappy.parquet => part-00029-5b54c5d5-bbcf-484d-a2ce-0d6f73df1a36-c000.snappy.parquet
    part-00030-5b54c5d5-bbcf-484d-a2ce-0d6f73df1a36-c000.snappy.parquet => part-00030-5b54c5d5-bbcf-484d-a2ce-0d6f73df1a36-c000.snappy.parquet
    part-00031-5b54c5d5-bbcf-484d-a2ce-0d6f73df1a36-c000.snappy.parquet => part-00031-5b54c5d5-bbcf-484d-a2ce-0d6f73df1a36-c000.snappy.parquet
Continue? [Y/n]: Y
Transfer starting (Tip: Enable auto-confirmation with `skyplane config set autoconfirm true`)

Storing debug information for transfer in /tmp/skyplane/transfer_logs/20221018_213223/client.log
✓ Initializing cloud keys (5/5) in 3.15s
✓ Provisioning gateway instances (2/2) in 34.83s
✓ Installing gateway package (2/2) in 13.12s
🚀 20.16GB transfer job launched
  Transfer progress (completing multi-part uploads) ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.2/20.2 GiB 128.4 MB/s 0:00:00
Compression saved 0.21% of egress fees
✓ Deprovisioning instances (2/2) in 2.82s

✅ Transfer completed successfully
Transfer runtime: 65.20s, Throughput: 2.47Gbps

```


## 💻 Terminate your cluster!

Finally, just to make sure that we don't have any instances running that might be burning up money, let's quickly deprovision everything.

💻 `skyplane deprovision`

```
No instances to deprovision
✓ Removing IPs from VPCs (4/4) in 2.05s

```

## 🎉 Congratulations! Your plane has now landed. Skyplane is an open sourced project. Feel free to use Skyplane for all your data mobility needs!


Acknowledgement: Thanks to [Skypilot](https://github.com/romilbhardwaj/skypilot-tutorial/) for the notebook template.